In [1]:
import nasdaqdatalink as quandl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import date, timedelta
from common import date_util
from data import investment_universe, sharadar_fundamentals, sharadar_prices, sharadar_tickers
from signals import fundamental_signal

In [2]:
quandl.ApiConfig.api_key = 'NRvcyMwNMXZ2ooDSM3nw'
factor_summary = {}
factor_pnl = {}
factor_list = ["accruals", "debt_to_equity", "net_profit_margin", "return_on_equity", #"gross_margin", "margin", "operating_margin", "operating_leverage",  "return_on_invcap",
               "book_to_price", "dividend_yield", "earnings_yield", "fcf_to_ev", "fcf_yield",
               "fcf_to_ic_growth", "size", "ncf_to_ev", "sales_to_price", "tangible_asset_to_price"]#, "ncfo_to_ev", "ncfo_to_mc", "ncf_to_mc"]

In [4]:
for factor in factor_list:
    pnl = []
    sector_level_pnl = {}
    dates = []
    rebal_date = date(2000, 1, 1)
    while rebal_date <= date(2021, 12, 31):
        rebal_date = date_util.get_next_rebal_day(rebal_date, 3)

        universe = investment_universe.get_SPX(rebal_date)
        fundamentals = sharadar_fundamentals.get_fundamentals(universe['ticker'].to_list(), rebal_date)
        p_fundamentals = sharadar_fundamentals.get_fundamentals(universe['ticker'].to_list(), rebal_date - timedelta(weeks=52))
        prices = sharadar_prices.get_prices(universe['ticker'].to_list(), rebal_date)
        price_df = prices[["ticker", "closeadj"]]
        exit_prices = sharadar_prices.get_prices(universe['ticker'].to_list(), date_util.get_next_rebal_day(rebal_date, 3))
        exit_prices_df = exit_prices[["ticker", "closeadj"]]
        sectors = sharadar_tickers.get_tickers(universe['ticker'].to_list(), rebal_date)
        sectors = sectors[["ticker", "sector", "industry"]]

        fundamentals = pd.merge(fundamentals, sectors, left_on="ticker", right_on="ticker", how="inner")
        fundamentals = pd.merge(fundamentals, price_df, left_on="ticker", right_on="ticker", how="inner")
        fundamentals = pd.merge(fundamentals, exit_prices_df, left_on="ticker", right_on="ticker", how="inner", suffixes=["", "_f1w"])
        fundamentals["forward_return"] = fundamentals["closeadj_f1w"] / fundamentals["closeadj"] - 1
        fundamentals = pd.merge(fundamentals, p_fundamentals, left_on="ticker", right_on="ticker", how="inner", suffixes=["", "_1y"])
        fundamentals.loc[fundamentals["sector"] == "Real Estate", "sector"] = "Financial Services"
        func = getattr(fundamental_signal, factor)
        fundamentals = func(fundamentals)
        fundamentals = fundamentals[~fundamentals[factor].isna()]
        fundamentals["composite"] = fundamentals.groupby("sector", group_keys=False)[factor].apply(lambda x: (x - np.mean(x)) / np.std(x))

        dates.append(rebal_date)
        if fundamentals.empty:
            pnl.append(0)
            continue

        fundamentals = fundamentals.sort_values("composite", ascending=False)
        fundamentals = fundamentals[~fundamentals["sector"].isin(["Real Estate"])]
        pnl.append(fundamentals[fundamentals["composite"] >= fundamentals.groupby("sector")["composite"].transform("quantile", 0.75)]["forward_return"].mean() - 
                fundamentals[fundamentals["composite"] <= fundamentals.groupby("sector")["composite"].transform("quantile", 0.25)]["forward_return"].mean())
        longs = fundamentals[fundamentals["composite"] >= fundamentals.groupby("sector")["composite"].transform("quantile", 0.8)].groupby("sector").agg({"forward_return": "sum", "ticker": "count"})
        shorts = fundamentals[fundamentals["composite"] <= fundamentals.groupby("sector")["composite"].transform("quantile", 0.2)].groupby("sector").agg({"forward_return": "sum", "ticker": "count"})
        sector_pnl = pd.merge(longs, shorts, left_index=True, right_index=True, suffixes=["_l", "_s"])
        sector_pnl["forward_return"] = sector_pnl["forward_return_l"] / sector_pnl["ticker_l"] - sector_pnl["forward_return_s"] / sector_pnl["ticker_s"]
        sector_level_pnl[rebal_date] = sector_pnl["forward_return"].to_list()

    sector_level_pnl_df = pd.DataFrame.from_dict(sector_level_pnl, orient='index')
    factor_array = []
    pnl = np.array(pnl)
    pnl[np.isnan(pnl)] = 0
    
    pnl_df = pd.DataFrame({"date": dates, "return": pnl})
    pnl_df["date"] = pd.to_datetime(pnl_df["date"])
    pnl_df.set_index("date", inplace=True)

    factor_array.append(np.mean(pnl) * 52)
    factor_array.append(np.std(pnl) * np.sqrt(52))
    sr = np.mean(pnl) / np.std(pnl) * np.sqrt(52)
    factor_array.append(sr)
    factor_array.append(sr * (1 + sr * pnl_df["return"].skew() / 6 - sr ** 2 * pnl_df["return"].kurt() / 24))

    cum_returns = np.cumprod(pnl + 1) - 1
    high_watermark = np.maximum.accumulate(cum_returns)
    drawdown = (high_watermark - cum_returns) / (1 + high_watermark)
    factor_array.append(np.max(drawdown))
    
    factor_array.append(len(pnl[pnl >= 0]) / len(pnl))
    factor_array.append(np.mean(pnl[pnl >= 0]))
    factor_array.append(np.mean(pnl[pnl < 0]))
    factor_array.append(np.max(pnl))
    factor_array.append(np.min(pnl))

    factor_array.append(np.mean(pnl_df.loc["2000":"2009", "return"]) * 52)
    factor_array.append(np.std(pnl_df.loc["2000":"2009", "return"]) * np.sqrt(52))
    factor_array.append(np.mean(pnl_df.loc["2000":"2009", "return"]) / np.std(pnl_df.loc["2000":"2009", "return"]) * np.sqrt(52))

    factor_array.append(np.mean(pnl_df.loc["2010":"2019", "return"]) * 52)
    factor_array.append(np.std(pnl_df.loc["2010":"2019", "return"]) * np.sqrt(52))
    factor_array.append(np.mean(pnl_df.loc["2010":"2019", "return"]) / np.std(pnl_df.loc["2010":"2019", "return"]) * np.sqrt(52))

    factor_array.extend(sector_level_pnl_df.sum(axis=0).to_list())
    factor_summary[factor] = factor_array
    factor_pnl[factor] = pnl

In [5]:
pd.DataFrame.from_dict(factor_summary, orient='index').to_csv("factor_summary.csv")

In [23]:
pd.DataFrame.from_dict(factor_pnl).to_csv("/Users/weizhang/Documents/_GIT/quant-strategies/data/PnL/pnl.csv")

In [28]:
pnl_df = pd.read_csv("/Users/weizhang/Documents/_GIT/quant-strategies/data/PnL/pnl.csv")
pnl_df.set_index("date", inplace=True)

In [31]:
pnl_df.corr()

,accruals,debt_to_equity,net_profit_margin,return_on_equity,book_to_price,dividend_yield,earnings_yield,fcf_to_ev,fcf_yield,fcf_to_ic_growth,momentum,size,ncf_to_ev,sales_to_price,tangible_asset_to_price
accruals,1.000000,0.297580,-0.356787,-0.263709,0.398182,0.256897,0.000202,0.223177,0.317022,0.087332,-0.198078,0.416354,0.042067,-0.073572,0.188680
debt_to_equity,0.297580,1.000000,-0.522291,-0.527674,0.604096,0.261836,-0.215365,-0.100540,0.130833,-0.104137,-0.286766,0.541011,-0.045972,-0.406603,0.008468
net_profit_margin,-0.356787,-0.522291,1.000000,0.859809,-0.786928,-0.023720,0.440283,0.253974,0.047416,0.157221,0.452552,-0.768533,0.084077,0.260979,-0.206462
return_on_equity,-0.263709,-0.527674,0.859809,1.000000,-0.786092,-0.029162,0.566553,0.305490,0.074945,0.163710,0.410355,-0.731120,0.133767,0.482528,-0.099511
book_to_price,0.398182,0.604096,-0.786928,-0.786092,1.000000,0.272296,-0.102857,0.011231,0.250655,-0.129289,-0.487894,0.808936,-0.101841,-0.375535,0.218670
dividend_yield,0.256897,0.261836,-0.023720,-0.029162,0.272296,1.000000,0.320992,0.327386,0.377709,-0.009102,-0.193682,0.161140,-0.021964,-0.108064,-0.108316
earnings_yield,0.000202,-0.215365,0.440283,0.566553,-0.102857,0.320992,1.000000,0.548459,0.433721,0.095560,0.062578,-0.214273,0.005349,0.351070,0.028360
fcf_to_ev,0.223177,-0.100540,0.253974,0.305490,0.011231,0.327386,0.548459,1.000000,0.906761,0.230928,0.015671,-0.079272,0.238991,0.282433,0.072897
fcf_yield,0.317022,0.130833,0.047416,0.074945,0.250655,0.377709,0.433721,0.906761,1.000000,0.195339,-0.094984,0.120379,0.211188,0.110319,0.112906
fcf_to_ic_growth,0.087332,-0.104137,0.157221,0.163710,-0.129289,-0.009102,0.095560,0.230928,0.195339,1.000000,0.194083,-0.128543,0.153481,0.104693,0.056364
